# Predicting Youtube View Count in 2007/2008

### Introduction

After being acquired by Google in 2006, Youtube obtained a rapid growth in popularity resulting in a growing community. The goal of this project is to use the data of 2007 to predict the view of videos in 2008. We will be using datasets from https://netsg.cs.sfu.ca/youtubedata/. The dataset uses a crawler to obtained video API and scrape the webpage for the remaining information.


### Methods

In [10]:
#Read the data
data2007 <- read.csv("dsci-310-group-13/data/datatable2007.txt")


[1] "C:/Users/Maggie/School"